<a href="https://colab.research.google.com/github/Shruti12377/gardevoir/blob/main/PRODIGY_ML_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Unzip the dataset
with zipfile.ZipFile('/path/to/food-101.zip', 'r') as zip_ref:
    zip_ref.extractall('/path/to/destination')

# Define paths
base_dir = '/path/to/destination/food-101'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dense(101, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


In [ ]:
# Example calorie mapping
calorie_mapping = {
    'apple_pie': 237,
    'baby_back_ribs': 292,
    # Add all 101 categories with their respective calorie content
    # ...
}

def estimate_calories(predicted_class):
    return calorie_mapping.get(predicted_class, "Calorie data not available")


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_food_and_calories(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    predictions = model.predict(img_array)
    predicted_class_idx = np.argmax(predictions, axis=1)[0]
    predicted_class = list(train_generator.class_indices.keys())[predicted_class_idx]

    estimated_calories = estimate_calories(predicted_class)
    return predicted_class, estimated_calories

# Test with an example image
img_path = '/path/to/example_image.jpg'
predicted_class, estimated_calories = predict_food_and_calories(img_path)
print(f"Predicted food: {predicted_class}, Estimated calories: {estimated_calories}")
